# 爬取雅思作文内容和评分数据

In [1]:
# 导入需要的模块
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from io import StringIO
from functools import partial

import time
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

### 初始化数据集

In [6]:
df = pd.DataFrame(columns=['problem','article','subParagraph','TR','CC','LR','GA','score'])
tmp = pd.Series({'problem': 0,'article': 0,'subParagraph': 0,'TR': 0,'CC': 0,'LR': 0,'GA': 0,'score': 0})

In [3]:
df = pd.read_csv("data.csv")
df

,problem,article,subParagraph,TR,CC,LR,GA,score,score
0,Some children spend hours every day on their s...,"In contemporary society, it is a common sight ...",4,9.0,9.0,8.5,9.0,9.0,NaN
1,Some children spend hours every day on their s...,The ubiquitous presence of smartphones has pro...,5,9.0,9.0,8.5,9.0,9.0,NaN
2,Some children spend hours every day on their s...,The proliferation of pocket-sized electronics ...,5,9.0,9.0,9.0,9.0,9.0,NaN
3,Some children spend hours every day on their s...,The phenomenon of children spending extensive ...,5,9.0,9.0,9.0,9.0,9.0,NaN
4,Some children spend hours every day on their s...,"In the contemporary era, the proliferation of ...",5,7.0,7.0,8.0,8.0,7.5,NaN
...,...,...,...,...,...,...,...,...,...
32030,The growth of online shopping will oneday lead...,Shops in both rural and urban areas will somed...,4,5.0,4.5,4.5,4.5,4.5,NaN
32031,Some people believe that climate affects a per...,Climate profoundly influences both personal li...,4,6.5,7.5,7.5,7.5,7.5,NaN
32032,Some people believe that climate affects a per...,limate has influence on both the personal life...,4,5.0,4.0,4.0,4.0,4.5,NaN
32033,Do you agree/disagree that teenagers should wo...,Some individuals believes that youth must work...,4,5.0,4.0,4.0,4.0,4.5,NaN


### 爬取数据

In [9]:
import os
os.system('export https_proxy=http://127.0.0.1:10809 http_proxy=http://127.0.0.1:10809 all_proxy=socks5://127.0.0.1:10809')

driver = webdriver.Edge()
url='https://engnovate.com/ugc-ielts-writing-task-2-topics/'
driver.get(url)

In [20]:
wait = WebDriverWait(driver, 20)
next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="pagination"]//a[@class="next page-numbers"]')))
next_button.click()

In [10]:
l = 0
i = 0 
count = 0
page = 340
while True:
    print("page : ", page)
    # 定位所有雅思作文话题
    topics = driver.find_elements(By.TAG_NAME, 'article')
    l = len(topics)
    i = 0
    while i < l:
        topics = driver.find_elements(By.TAG_NAME, 'article')
        if i >= len(topics):
            break
        topic = topics[i]
        try:
            link = topic.find_element(By.CLASS_NAME,'show-samples-button')
            link.click()
            # 记录作文命题
            link = driver.find_element(By.CLASS_NAME, "entry-header")
            problem = link.find_element(By.TAG_NAME, "h1").text
            # 遍历文章
            articles = driver.find_elements(By.TAG_NAME, 'article')
            for article in articles:
                tmp = pd.Series({'problem': problem,'article': 0,'subParagraph': 0,'TR': 0,'CC': 0,'LR': 0,'GA': 0,'score': 0})
                try:
                    link = article.find_element(By.CLASS_NAME,'read-more-button')
                    link = link.find_element(By.TAG_NAME,'a')
                    link.click()
                except Exception as e:
                    print(f"Error clicking link in article.")
                    continue
                # 读取作文文本
                try:
                    paragraphs = driver.find_elements(By.CSS_SELECTOR, '.entry-content p')
                except Exception as e:
                    print(f"Error getting writing text.")
                    driver.back()
                    continue
                text = ''
                cnt = 0
                for paragraph in paragraphs:
                    text += paragraph.text
                    cnt += 1
                tmp['article'] = text
                tmp['subParagraph'] = cnt

                # 读取分数
                time.sleep(3)
                try:
                    tmp['score'] = float(driver.find_element(By.ID, "overall-band-score").text)
                    tmp['TR'] = float(driver.find_element(By.ID, "task-response-score").text)
                    tmp['CC'] = float(driver.find_element(By.ID, "coherence-cohesion-score").text)
                    tmp['LR'] = float(driver.find_element(By.ID, "lexical-resource-score").text)
                    tmp['GA'] = float(driver.find_element(By.ID, "grammatical-range-accuracy-score").text)
                except Exception as e:
                    driver.back()
                    print(f"Error getting scores.")
                    continue
                df = df.append(tmp, ignore_index=True)
                count += 1
                print(count,' : ',tmp['score'],tmp['TR'],tmp['CC'],tmp['LR'],tmp['GA'])

                driver.back()

            driver.back()
            i += 1

        except Exception as e:
            print(f"Error clicking link in topic.")

    # 翻页
    try:
        wait = WebDriverWait(driver, 20)
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="pagination"]//a[@class="next page-numbers"]')))
        next_button.click()
    except Exception as e:
        print(f"Error clicking 'Next' button.")
        break  # 如果没有找到Next按钮或发生错误，结束爬取

    # 等待页面加载
    wait.until(lambda driver: driver.title)  # 确保页面已加载
    page += 1

# 完成后关闭浏览器
driver.close()

page :  340
1  :  7.0 7.0 7.0 7.0 6.0
2  :  6.5 7.0 6.0 7.0 6.0
3  :  5.0 6.0 5.0 5.5 3.5
4  :  7.5 8.0 7.0 7.0 8.0
5  :  4.0 4.5 3.5 4.5 3.5
6  :  5.0 6.0 4.5 5.5 4.5
7  :  5.0 5.5 4.5 5.0 4.0
8  :  8.0 7.0 8.0 8.0 8.0
9  :  5.0 6.5 5.0 5.5 3.5
10  :  6.5 6.0 6.5 6.5 6.5
11  :  4.5 5.0 4.0 4.0 4.0
12  :  7.5 8.0 7.5 7.5 7.5
13  :  8.0 8.0 8.0 7.5 8.0
14  :  6.5 7.0 7.0 6.5 6.0
15  :  7.0 8.0 6.5 6.5 6.0
16  :  4.5 5.5 4.0 5.0 4.0
17  :  6.5 6.5 5.5 6.5 6.5
18  :  6.0 6.0 5.0 6.0 6.0
19  :  6.5 7.5 5.5 6.5 6.5
20  :  5.5 6.5 5.0 5.0 5.0
21  :  5.0 5.5 4.5 5.0 4.0
22  :  6.5 6.0 6.0 7.0 7.0
23  :  7.0 7.5 7.0 7.0 7.0
24  :  7.0 7.5 7.0 7.0 7.0
25  :  5.0 5.0 5.0 5.0 5.0
26  :  4.5 5.0 4.5 4.5 4.5
27  :  8.0 7.0 8.0 8.0 8.0
28  :  8.0 7.0 8.0 8.0 8.0
29  :  5.5 6.0 5.0 5.0 6.0
30  :  5.5 6.0 5.0 5.0 6.0
31  :  7.0 7.5 7.0 7.0 7.0
32  :  4.5 5.0 4.0 5.0 4.0
page :  341
33  :  4.5 4.5 3.5 4.5 4.5
34  :  4.5 5.0 4.0 4.0 4.0
35  :  8.0 8.0 7.5 8.5 8.5
36  :  7.5 7.5 7.5 7.5 7.5
37  :  5.5 6.

In [11]:
df

,problem,article,subParagraph,TR,CC,LR,GA,score,score
0,Some children spend hours every day on their s...,"In contemporary society, it is a common sight ...",4,9.0,9.0,8.5,9.0,9.0,NaN
1,Some children spend hours every day on their s...,The ubiquitous presence of smartphones has pro...,5,9.0,9.0,8.5,9.0,9.0,NaN
2,Some children spend hours every day on their s...,The proliferation of pocket-sized electronics ...,5,9.0,9.0,9.0,9.0,9.0,NaN
3,Some children spend hours every day on their s...,The phenomenon of children spending extensive ...,5,9.0,9.0,9.0,9.0,9.0,NaN
4,Some children spend hours every day on their s...,"In the contemporary era, the proliferation of ...",5,7.0,7.0,8.0,8.0,7.5,NaN
...,...,...,...,...,...,...,...,...,...
33100,People should never eat meat because raising a...,There are too many people who eat meat. This l...,4,6.0,5.0,5.0,5.0,5.5,NaN
33101,Some people think that companies should provid...,Some parts pf society consider exercise time o...,3,5.5,4.5,5.0,4.0,5.0,NaN
33102,Some people think that companies should provid...,Several people's opinion is that enterprises s...,4,4.0,4.0,5.0,4.0,4.5,NaN
33103,Robotics Mapping and Simulation using Python a...,Lab Report1 — Abstract:In this task I combined...,22,7.0,6.0,5.0,6.0,6.0,NaN


In [12]:
print("Duplicated rows: {}".format(df.duplicated(subset=None, keep='first').sum()))
df = df[~df.duplicated(subset=None, keep='first')]

Duplicated rows: 58


In [13]:
len(df)

33047

In [14]:
df.to_csv("data.csv",index=False)

In [15]:
page

306

In [10]:
df.describe()

,subParagraph,TR,CC,LR,GA,score,score
count,31161.000000,31161.000000,31161.000000,31161.000000,31161.000000,30888.000000,273.000000
mean,4.202657,6.513896,5.882773,6.060871,5.825497,6.142330,6.540293
std,1.332559,1.185580,1.466366,1.336034,1.602915,1.318312,1.480333
min,1.000000,3.000000,2.500000,0.000000,3.000000,4.000000,4.000000
25%,4.000000,5.500000,4.500000,5.000000,4.500000,5.000000,5.500000
50%,4.000000,6.500000,6.000000,6.000000,5.500000,6.000000,6.500000
75%,4.000000,7.000000,7.000000,7.000000,7.000000,7.000000,8.000000
max,49.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
